In [2]:
import pandas as pd
df = pd.read_csv('../0-data/Root_Insurance_data.csv')


In [ ]:
Check data

In [3]:
df.head()

,Currently Insured,Number of Vehicles,Number of Drivers,Marital Status,bid,rank,click,policies_sold
0,unknown,2,1,M,10.0,2,False,0
1,Y,3,1,M,10.0,5,False,0
2,Y,3,2,M,10.0,5,False,0
3,N,3,2,S,10.0,4,False,0
4,unknown,2,2,S,10.0,2,False,0


In [4]:
df.corr()

,Number of Vehicles,Number of Drivers,bid,rank,click,policies_sold
Number of Vehicles,1.000000,-0.029474,NaN,0.399393,-0.171822,-0.130380
Number of Drivers,-0.029474,1.000000,NaN,-0.076289,0.021604,-0.005525
bid,NaN,NaN,NaN,NaN,NaN,NaN
rank,0.399393,-0.076289,NaN,1.000000,-0.416046,-0.262496
click,-0.171822,0.021604,NaN,-0.416046,1.000000,0.606136
policies_sold,-0.130380,-0.005525,NaN,-0.262496,0.606136,1.000000
